In [10]:
#exp 1

from mpi4py import MPI
import numpy as np



def parallel_dfs(graph, start_node):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    visited = set()
    stack = [start_node]
    result = []

    while stack:
        node = stack.pop()
        if node not in visited:
            visited.add(node)
            result.append(node)
            neighbors = graph[node]

            # Distribute the neighbors across processes
            chunk_size = (len(neighbors) + size - 1) // size
            chunk = neighbors[rank * chunk_size:(rank + 1) * chunk_size]

            for neighbor in chunk:
                if neighbor not in visited:
                    stack.append(neighbor)

        # Gather results from all processes
        result = comm.gather(result, root=0)

        if rank == 0:
            result = [node for sublist in result for node in sublist]

    return result

def parallel_bfs(graph, start_node):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    visited = set()
    queue = [start_node]
    result = []

    while queue:
        node = queue.pop(0)
        if node not in visited:
            visited.add(node)
            result.append(node)
            neighbors = graph[node]

            # Distribute the neighbors across processes
            chunk_size = (len(neighbors) + size - 1) // size
            chunk = neighbors[rank * chunk_size:(rank + 1) * chunk_size]

            for neighbor in chunk:
                if neighbor not in visited:
                    queue.append(neighbor)

        # Gather results from all processes
        result = comm.gather(result, root=0)

        if rank == 0:
            result = [node for sublist in result for node in sublist]

    return result





# Define the graph for DFS and BFS
graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['F'],
    'F': []
}

# Perform parallel DFS
dfs_result = parallel_dfs(graph, 'A')
print("DFS Result:", dfs_result)

# Perform parallel BFS
bfs_result = parallel_bfs(graph, 'A')
print("BFS Result:", bfs_result)







ImportError: cannot import name 'omp' from 'mpi4py' (c:\Users\deven\anaconda3\lib\site-packages\mpi4py\__init__.py)

In [2]:
# exp 2
from mpi4py import MPI
import numpy as np

def parallel_bubble_sort(data):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    n = len(data)
    local_data = data[rank * (n // size):(rank + 1) * (n // size)]
    sorted_data = comm.gather(sorted(local_data), root=0)

    if rank == 0:
        sorted_data = [item for sublist in sorted_data for item in sublist]
        # Perform final bubble sort on root process
        for i in range(n):
            for j in range(n - i - 1):
                if sorted_data[j] > sorted_data[j + 1]:
                    sorted_data[j], sorted_data[j + 1] = sorted_data[j + 1], sorted_data[j]

    return sorted_data

def parallel_merge_sort(data):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    n = len(data)

    def merge_sort(arr):
        if len(arr) <= 1:
            return arr

        mid = len(arr) // 2
        left = arr[:mid]
        right = arr[mid:]

        left = merge_sort(left)
        right = merge_sort(right)

        return merge(left, right)

    def merge(left, right):
        merged = []
        i, j = 0, 0

        while i < len(left) and j < len(right):
            if left[i] <= right[j]:
                merged.append(left[i])
                i += 1
            else:
                merged.append(right[j])
                j += 1

        while i < len(left):
            merged.append(left[i])
            i += 1

        while j < len(right):
            merged.append(right[j])
            j += 1

        return merged

    local_data = data[rank * (n // size):(rank + 1) * (n // size)]
    local_sorted_data = merge_sort(local_data)
    sorted_data = comm.gather(local_sorted_data, root=0)

    if rank == 0:
        sorted_data = merge_sort([item for sublist in sorted_data for item in sublist])

    return sorted_data

# Perform parallel bubble sort
data = [7, 3, 1, 5, 2, 4, 6]
bubble_sorted_data = parallel_bubble_sort(data)
print("Bubble Sort Result:", bubble_sorted_data)

# Perform parallel merge sort
merge_sorted_data = parallel_merge_sort(data)
print("Merge Sort Result:", merge_sorted_data)


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\deven\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
# exp 3
from mpi4py import MPI
import numpy as np

def parallel_reduction_min_max_sum_avg(data):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    # Scatter the data across processes
    local_data = np.array_split(data, size)[rank]

    # Perform local reduction to find local min, max, sum, and count
    local_min = np.min(local_data)
    local_max = np.max(local_data)
    local_sum = np.sum(local_data)
    local_count = len(local_data)

    # Perform global reduction using MPI reduce operations
    global_min = comm.reduce(local_min, op=MPI.MIN, root=0)
    global_max = comm.reduce(local_max, op=MPI.MAX, root=0)
    global_sum = comm.reduce(local_sum, op=MPI.SUM, root=0)
    global_count = comm.reduce(local_count, op=MPI.SUM, root=0)

    if rank == 0:
        global_avg = global_sum / global_count
        return global_min, global_max, global_sum, global_avg

    return None

# Example usage

if __name__ == "__main__":
    data = np.array([7, 3, 1, 5, 2, 4, 6])

    min_val, max_val, sum_val, avg_val = parallel_reduction_min_max_sum_avg(data)

    if MPI.COMM_WORLD.Get_rank() == 0:
        print("Minimum:", min_val)
        print("Maximum:", max_val)
        print("Sum:", sum_val)
        print("Average:", avg_val)


Minimum: 1
Maximum: 7
Sum: 28
Average: 4.0


In [3]:
import omp

def bubble_sort(array):
  # Parallelize the following loop over 4 threads.
  with omp.parallel():
    for i in range(len(array) - 1):
      for j in range(len(array) - i - 1):
        if array[j] > array[j + 1]:
          # Swap the elements if they are in the wrong order.
          array[j], array[j + 1] = array[j + 1], array[j]

def main():
  # Create an array of 10 random numbers.
  array = [randint(0, 100) for i in range(10)]

  # Print the original array.
  print("Original array:", array)

  # Sort the array using parallel bubble sort.
  bubble_sort(array)

  # Print the sorted array.
  print("Sorted array:", array)


main()

ModuleNotFoundError: No module named 'omp'

In [4]:
!pip install omp

ERROR: Could not find a version that satisfies the requirement omp (from versions: none)
ERROR: No matching distribution found for omp


In [1]:
import multiprocessing as mp
from random import randint

def bubble_sort(array):
  # Parallelize the following loop over 4 threads.
  with mp.Pool(4) as pool:
    for i in range(len(array) - 1):
      for j in range(len(array) - i - 1):
        if array[j] > array[j + 1]:
          # Swap the elements if they are in the wrong order.
          array[j], array[j + 1] = array[j + 1], array[j]

def main():
  # Create an array of 10 random numbers.
  array = [randint(0, 100) for i in range(10)]

  # Print the original array.
  print("Original array:", array)

  # Sort the array using parallel bubble sort.
  bubble_sort(array)

  # Print the sorted array.
  print("Sorted array:", array)

main()

Original array: [17, 72, 99, 35, 43, 29, 54, 74, 45, 2]
Sorted array: [2, 17, 29, 35, 43, 45, 54, 72, 74, 99]


In [3]:
import multiprocessing as mp

def merge_sort(array):
  # Check if the array is empty or has only one element.
  if len(array) <= 1:
    return array

  # Split the array into two halves.
  middle = len(array) // 2
  left = merge_sort(array[:middle])
  right = merge_sort(array[middle:])

  # Merge the two halves.
  return merge(left, right)

def merge(left, right):
  # Create a new array to store the merged results.
  merged = []

  # Iterate over the left and right arrays.
  with mp.Pool(4) as pool:
    i, j = 0, 0
    while i < len(left) and j < len(right):
      # If the current element in the left array is less than or equal to the current element in the right array, add it to the merged array.
      if left[i] <= right[j]:
        merged.append(left[i])
        i += 1
      else:
        merged.append(right[j])
        j += 1

    # Add the remaining elements from the left and right arrays to the merged array.
    merged += left[i:]
    merged += right[j:]

  # Return the merged array.
  return merged

def main():
  # Create an array of 10 random numbers.
  array = [randint(0, 100) for i in range(10)]

  # Print the original array.
  print("Original array:", array)

  # Sort the array using parallel merge sort.
  sorted_array = merge_sort(array)

  # Print the sorted array.
  print("Sorted array:", sorted_array)

main()

Original array: [93, 93, 87, 88, 4, 10, 99, 14, 89, 72]
Sorted array: [4, 10, 14, 72, 87, 88, 89, 93, 93, 99]


In [8]:
import mpi4py 
